<a href="https://colab.research.google.com/github/JibKh/SPROJ-Hajj/blob/master/SPROJ_Hajj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check GPU

The GPU must NOT be Tesla K80. If it is then factory reset runtime and try again.

In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
Mon May 10 09:50:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |         

# User Input

In [2]:
# If you have a video put 1. If you have frames put 2.
# Based on your choice, update the below cells accordingly
video_frames = 1

# If you would like to visualize the flow frames. This WILL DELETE ALL THE FLOWFRAMES GENERATED TO SAVE STORAGE.
visualize = False 

In [3]:
# Video Input

if video_frames == 1:

  video_name = 'test2.mp4' # If you have a video you want to run inference on. Please include .mp4 or whatever extension the video has.
  video_local_gdrive = 2 # If you want to upload a video from your local drive, choose 1. If from your google drive, choose 2. If some other option, go to section "Upload Video".
  
  # If your video is on gdrive, please add Gdrive ID here.
  if video_local_gdrive == 2:
    video_gdrive = '1lT1MDLqteLpRld10q2cWhdf3Erzrj9O9' # File_id for your google drive video. Use this link to see how to get file ID https://docs.meiro.io/books/meiro-integrations/page/where-can-i-find-the-file-id-on-google-drive#:~:text=To%20locate%20the%20File%20ID,%3D%60%20is%20the%20File%20ID.


In [4]:
# Frames Input

if video_frames == 2:

  # Mount Gdrive
  from google.colab import drive
  drive.mount('/content/drive')

  frames_zip_name = "2 - Arabic.zip" # If you have the frames, enter its zip file here. For ex: "3 - Video.zip"
  frames_directory = '../gdrive/My Drive/Hajj Videos/Frames/' # Where the frames are located. If its in gdrive: '../gdrive/My Drive/Location of Zip/' Change the Location of Zip to wherever yours is stored.


In [5]:
# Skip / Average Options

no_frames_skip = 1 # How many frames you want skipped. For eg if its 2 then from frames 1,2,3,4,5,6,7 we take frames 1,4,7. Leave at None to not skip frames.

# Only one can work at a time
no_average_frames = None # How many frames you want to average. Leave at None if you don't want to avg.
running_average = False # If you want to visually see running average

In [6]:
# DO NOT TOUCH

flow_video_name = 'flowvid.mp4'

# FOR VIDEOS
if video_frames == 1:

  # This will prompt you to upload the video from your local machine
  if video_local_gdrive == 1:

    from google.colab import files
    uploaded = files.upload()
    if video_name != list(uploaded.keys())[0]:
      video_name = list(uploaded.keys())[0]

  # This downloads the gdrive video
  elif video_local_gdrive == 2:

    # This will download the video
    !gdown --id $video_gdrive

# FOR FRAMES
if video_frames == 2:
  
  !mkdir -p ./frames
  unzip_file = frames_directory + frames_zip_name
  !unzip '$unzip_file' -d ./frames


Downloading...
From: https://drive.google.com/uc?id=1lT1MDLqteLpRld10q2cWhdf3Erzrj9O9
To: /content/test2.mp4
9.70MB [00:00, 18.9MB/s]


In [7]:
# # THIS WILL DELETE ALL THE FRAMES, FLOW FRAMES, OUTPUT FILES, FLOW VIDEO.
# # If you would like to keep something, make sure to comment it out and save it before running this.

# # To restart and run again. Change any parameters above and then go click the "User Input and Restart" cell. Then Runtime -> Run After.

# !rm -r ./frames
# !rm -r ./Flo
# !rm -r ./FlowFrames
# !rm -r ./output
# # !rm ./$video_name
# !rm -r ./Average_Frames
# !rm -r ./Running_Avg_Frames
# !rm -r ./FlowVideo
# !pip install setproctitle colorama scipy==1.1.0

# Setup Video / Frames

## Setup Video

Converting video to frames

In [8]:
import os

if video_frames == 1:
  def mkdir_ifnotexists(dir):
      if os.path.exists(dir):
          return
      os.mkdir(dir)

In [9]:
if video_frames == 1:
  vid_file = video_name
  frame_pth = './frames'
  mkdir_ifnotexists(frame_pth)
  cmd = "ffmpeg -i %s -start_number 0 -vsync 0 %s/frame_%%06d.png" % (
              vid_file,
              frame_pth,
          )
  os.system(cmd)

In [10]:
!rm test.mp4

rm: cannot remove 'test.mp4': No such file or directory


## Setup Frames

Rename Frames

In [11]:
!ls ./frames | wc -l # Use to recheck if number of frames is consistent and nothing went wrong.

460


In [12]:
if video_frames == 2:
  import os

  file_dir = "./frames/"
  for count, filename in enumerate(sorted(os.listdir(file_dir))):
    # print(filename)
    if filename[-11:] == "_UTC+01.jpg":
      src = file_dir + filename
      dst = file_dir + str(count).zfill(6) + '.png'
      os.rename(src, dst)

In [13]:
!ls ./frames | wc -l # Use to recheck if number of frames is consistent and nothing went wrong.

460


Skip Frames

In [14]:
!ls ./frames | wc -l # Use to recheck if number of frames is consistent and nothing went wrong.

460


In [15]:
if no_frames_skip != None:
  directory = "./frames"

  temp_skip = no_frames_skip

  for i, file in enumerate(sorted(os.listdir(directory))):
    if (temp_skip != 0) and (file[-4:] == ".png"):
      os.remove(directory + '/' + file)
      temp_skip = temp_skip - 1
      continue
    temp_skip = no_frames_skip


# if no_frames_skip != None:
#   directory = './frames'
#   # no_frames_skip = 2

#   for i, file in enumerate(sorted(os.listdir(directory))):
#     if (file[-4:] == ".png") and (int(file[0:-4]) % no_frames_skip == 0):
#       # print(file)
#       os.remove(directory+'/' + file)

In [16]:
if no_frames_skip != None:
  import os

  file_dir = "./frames/"
  for count, filename in enumerate(sorted(os.listdir(file_dir))):
    # print(filename)
    if filename[-4:] == ".png":
      src = file_dir + filename
      dst = file_dir + str(count).zfill(6) + '.png'
      os.rename(src, dst)

In [17]:
!ls ./frames | wc -l # Use to recheck if number of frames is consistent and nothing went wrong.

230


# NWPU

## Resized Images

In [18]:
# FlowNet outputs frames in multiples of 64. So we ensure it matches that for NWPU also.

from PIL import Image

img = Image.open("./frames/000000.png")

# New Sizes
x = (img.size[0] // 64) * 64
y = (img.size[1] // 64) * 64

x, y

(1280, 704)

In [ ]:
dir = "./frames"

for frame in sorted(os.listdir(dir)):
  if frame[-3:] == 'png':

    # Open image and resize
    image = Image.open(dir + "/" + frame)
    new_image = image.resize((x, y))

    # Remove image and write new one
    os.remove(dir + "/" + frame)
    new_image.save(dir + "/" + frame)


## Setup text file

In [ ]:
f = open("./frames/test1.txt", "w")

i = 0
for frame in sorted(os.listdir("./frames")):

  if frame[-3:] == 'png':

    if i != 0:
      f.write("\n")
      
    f.write(frame[:-4])
    i += 1

f.close()

## Import NWPU

In [ ]:
!gdown --id '1-QsJ4EHwMBwDRQl7bMeCkI0gvUtM2eGZ'

In [ ]:
!unzip NWPU.zip
!rm NWPU.zip

## Do NWPU Stuff

In [ ]:
%cd NWPU-Crowd-Sample-Code/
!mkdir Final_Results

In [ ]:
# Edit dataRoot, result_path, model_path, txtpath variables in test.py
!python test.py

In [ ]:
!mv Final_Results /content/
%cd ../
!mv Final_Results NWPU_Results
!rm -r NWPU-Crowd-Sample-Code
!rm /content/frames/test1.txt

## To Zip NWPU files and then have to get shareable link

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd /content/drive/MyDrive/
# !zip -r NWPU.zip './NWPU-Crowd-Sample-Code'
# %cd ../../

# FlowNet

## Setup and Install FlowNet2

In [ ]:
!pip install torch==1.0.0 torchvision==0.2.2 -f https://download.pytorch.org/whl/cu100/torch_stable.html
!pip install pypng
!pip install tensorboardx
!pip install setproctitle colorama scipy==1.1.0
!pip install flowiz -U

In [ ]:
import os
# get flownet2-pytorch source
!git clone https://github.com/NVIDIA/flownet2-pytorch.git
!mv /content/flownet2-pytorch /content/flownet2pytorch
os.chdir('./flownet2pytorch')
# install custom layers
!bash install.sh

In [ ]:
import os
os.sys.path.append('/root/.local/lib/python3.6/site-packages/resample2d_cuda-0.0.0-py3.6-linux-x86_64.egg')
os.sys.path.append('/root/.local/lib/python3.6/site-packages/correlation_cuda-0.0.0-py3.6-linux-x86_64.egg')
os.sys.path.append( '/root/.local/lib/python3.6/site-packages/channelnorm_cuda-0.0.0-py3.6-linux-x86_64.egg')

In [ ]:
!python main.py --help

## Training and Validation - Not tested

If you do not want to train your model, you can skip this and move on to inference.

The dataset my team used is quite large and we have unlimited storage on OneDrive. So we have mounted OneDrive to read and write data to. <br>
To understand how to use it: https://www.youtube.com/watch?v=U6YPgARhRzA&t=255s&ab_channel=BoostUpStation

### OneDrive

In [ ]:
# !wget https://downloads.rclone.org/v1.50.1/rclone-v1.50.1-linux-amd64.deb
# !apt install ./rclone-v1.50.1-linux-amd64.deb

In [ ]:
# !rclone config

In [ ]:
# !sudo mkdir /content/onedrive
# !nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive &

### Train and Validate

In [ ]:
# !python main.py --batch_size 8 --model FlowNet2 --loss=L1Loss --optimizer=Adam --optimizer_lr=1e-4 \
# --training_dataset MpiSintelFinal --training_dataset_root /path/to/mpi-sintel/final/dataset  \
# --validation_dataset MpiSintelClean --validation_dataset_root /path/to/mpi-sintel/clean/dataset

## Run inference

Download the checkpoint. <br>
If you have your own checkpoint after training, skip this step.

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1hF8vS6YeHkx3j2pfCeQqqZGwA_PJq_Da',dest_path='./FlowNet2_checkpoint.pth.tar')

Run inference. <br>
You can learn more about each command from here: https://towardsdatascience.com/generating-optical-flow-using-nvidia-flownet2-pytorch-implementation-d7b0ae6f8320

In [ ]:
!python main.py --inference --model FlowNet2 --save_flow --save ./output --inference_dataset ImagesFromFolder --inference_dataset_root ../frames/ --resume ./FlowNet2_checkpoint.pth.tar

## Average

In [ ]:
if running_average == True or no_average_frames != None:
  import numpy as np
  from pathlib import Path
  import os
  from utils.flow_utils import writeFlow

  def write_frame_ra(flow, i):
    dir = "./Running_Avg_Frames/" + str(i).zfill(6) + ".flo"
    writeFlow(dir, flow)

  def write_frame_avg(flow, i):
    dir = "./Average_Frames/" + str(i).zfill(6) + ".flo"
    writeFlow(dir, flow)

  def average_list(list1):
    length_list = len(list1)
    temp_addition = 0
    for i in range(0, length_list):
        temp_addition += list1[i]
    temp_val = temp_addition / length_list
    return temp_val
    # try:
    #   return (list1[0] + list1[1]) / 2
    # except:
    #   print("Error in averaging")

  def make_flow(flo):
    tag = np.fromfile(flo, np.float32, count=1)[0]
    width = np.fromfile(flo, np.int32, count=1)[0]
    height = np.fromfile(flo, np.int32, count=1)[0]
    nbands = 2
    tmp = np.fromfile(flo, np.float32, count= nbands * width * height)
    flow = np.resize(tmp, (int(height), int(width), int(nbands)))
    return flow

  if no_average_frames != None:
    flow_list = [None] * no_average_frames
    index_flow = 0
    index_name = 0

  numerator = 0
  denominator = 0
  index_name = 0

  !mkdir ./Running_Avg_Frames
  !mkdir ./Average_Frames
  dir = './output/inference/run.epoch-0-flow-field/'

  for i, flo_file in enumerate(sorted(os.listdir(dir))):
    if flo_file[-3:] != "flo":
      continue

    path = Path(dir + flo_file)
    with path.open(mode='r') as flo:
      final_flo = make_flow(flo) # From their own code

      if running_average == True and no_average_frames == None:
        # Method 4
        if denominator == 0:
          numerator += final_flo
          denominator = 1
        else:
          numerator += final_flo
          denominator +=1

          average_flow = numerator/denominator

          write_frame(average_flow, index_name)
          index_name += 1
          
        # os.remove(dir + flo_file)

      # Method 3
      if running_average == False and no_average_frames != None:
        if (index_flow % no_average_frames == 0) and index_flow != 0:
          average_flow = average_list(flow_list)
          write_frame_ra(average_flow, index_name)
          index_flow = 0
          index_name += 1

        flow_list[index_flow] = final_flo
        index_flow += 1

        if i == len(os.listdir(dir)) - 1:
          average_flow = average_list(flow_list)
          write_frame_avg(average_flow, index_name)

        os.remove(dir + flo_file)

      # Method 1
      # if i == 0:
      #   flow_list[0] = flow
      # else:
      #   flow_list[1] = flow
      #   average_flow = average_list(flow_list)
      #   write_frame(average_flow, i-1)
      #   flow_list[0] = average_flow

      # Method 2
      # if index_flow == 0:
      #   flow_list[index_flow] = flow
      #   index_flow += 1
      # else:
      #   flow_list[index_flow] = flow
      #   index_flow = 0
      #   average_flow = average_list(flow_list)
      #   write_frame(average_flow, index_name)
      #   index_name += 1

      

## Visualizing

### Flowiz technique

In [ ]:
# if visualize == True:

#   !python -m flowiz \
#   ./Flo/*.flo \
#   -o FlowFrames \
#   -v FlowVideo \
#   -r 15

#   !mv ./FlowVideo/000000.flo.mp4 './FlowVideo/$flow_video_name'

In [ ]:
# from google.colab import files
# files.download('./FlowVideo/'+flow_video_name)

### Scipy Method

Install scipy as some tensorflow functionality requires updated scipy

In [ ]:
if visualize == True:
  import time

  start_time = time.time()

  !pip install scipy==1.4.1

Define show_flow() for visualization.
 Original Source https://github.com/sampepose/flownet2-tf/blob/master/src/flowlib.py

In [ ]:
# Source:https://github.com/sampepose/flownet2-tf/blob/master/src/flowlib.py
import matplotlib.pyplot as plt
import numpy as np

UNKNOWN_FLOW_THRESH = 1e7
def show_flow(filename):
    """
    visualize optical flow map using matplotlib
    :param filename: optical flow file
    :return: None
    """
    flow = read_flow(filename)
    img = flow_to_image(flow)
    plt.imshow(img)
    plt.show()

def read_flow(filename):
    """
    read optical flow from Middlebury .flo file
    :param filename: name of the flow file
    :return: optical flow data in matrix
    """
    f = open(filename, 'rb')
    magic = np.fromfile(f, np.float32, count=1)
    data2d = None

    if 202021.25 != magic:
        print ('Magic number incorrect. Invalid .flo file')
    else:
        w = int(np.fromfile(f, np.int32, count=1)[0])
        h = int(np.fromfile(f, np.int32, count=1)[0])
        #print("Reading %d x %d flo file" % (h, w))
        data2d = np.fromfile(f, np.float32, count=2 * w * h)
        # reshape data into 3D array (columns, rows, channels)
        data2d = np.resize(data2d, (h, w, 2))
    f.close()
    return data2d

def flow_to_image(flow):
    """
    Convert flow into middlebury color code image
    :param flow: optical flow map
    :return: optical flow image in middlebury color
    """
    u = flow[:, :, 0]
    v = flow[:, :, 1]

    maxu = -999.
    maxv = -999.
    minu = 999.
    minv = 999.

    idxUnknow = (abs(u) > UNKNOWN_FLOW_THRESH) | (abs(v) > UNKNOWN_FLOW_THRESH)
    u[idxUnknow] = 0
    v[idxUnknow] = 0

    maxu = max(maxu, np.max(u))
    minu = min(minu, np.min(u))

    maxv = max(maxv, np.max(v))
    minv = min(minv, np.min(v))

    rad = np.sqrt(u ** 2 + v ** 2)
    maxrad = max(-1, np.max(rad))

    #print( "max flow: %.4f\nflow range:\nu = %.3f .. %.3f\nv = %.3f .. %.3f" % (maxrad, minu,maxu, minv, maxv))

    u = u/(maxrad + np.finfo(float).eps)
    v = v/(maxrad + np.finfo(float).eps)

    img = compute_color(u, v)

    idx = np.repeat(idxUnknow[:, :, np.newaxis], 3, axis=2)
    img[idx] = 0

    return np.uint8(img)


def compute_color(u, v):
    """
    compute optical flow color map
    :param u: optical flow horizontal map
    :param v: optical flow vertical map
    :return: optical flow in color code
    """
    [h, w] = u.shape
    img = np.zeros([h, w, 3])
    nanIdx = np.isnan(u) | np.isnan(v)
    u[nanIdx] = 0
    v[nanIdx] = 0

    colorwheel = make_color_wheel()
    ncols = np.size(colorwheel, 0)

    rad = np.sqrt(u**2+v**2)

    a = np.arctan2(-v, -u) / np.pi

    fk = (a+1) / 2 * (ncols - 1) + 1

    k0 = np.floor(fk).astype(int)

    k1 = k0 + 1
    k1[k1 == ncols+1] = 1
    f = fk - k0

    for i in range(0, np.size(colorwheel,1)):
        tmp = colorwheel[:, i]
        col0 = tmp[k0-1] / 255
        col1 = tmp[k1-1] / 255
        col = (1-f) * col0 + f * col1

        idx = rad <= 1
        col[idx] = 1-rad[idx]*(1-col[idx])
        notidx = np.logical_not(idx)

        col[notidx] *= 0.75
        img[:, :, i] = np.uint8(np.floor(255 * col*(1-nanIdx)))

    return img


def make_color_wheel():
    """
    Generate color wheel according Middlebury color code
    :return: Color wheel
    """
    RY = 15
    YG = 6
    GC = 4
    CB = 11
    BM = 13
    MR = 6

    ncols = RY + YG + GC + CB + BM + MR

    colorwheel = np.zeros([ncols, 3])

    col = 0

    # RY
    colorwheel[0:RY, 0] = 255
    colorwheel[0:RY, 1] = np.transpose(np.floor(255*np.arange(0, RY) / RY))
    col += RY

    # YG
    colorwheel[col:col+YG, 0] = 255 - np.transpose(np.floor(255*np.arange(0, YG) / YG))
    colorwheel[col:col+YG, 1] = 255
    col += YG

    # GC
    colorwheel[col:col+GC, 1] = 255
    colorwheel[col:col+GC, 2] = np.transpose(np.floor(255*np.arange(0, GC) / GC))
    col += GC

    # CB
    colorwheel[col:col+CB, 1] = 255 - np.transpose(np.floor(255*np.arange(0, CB) / CB))
    colorwheel[col:col+CB, 2] = 255
    col += CB

    # BM
    colorwheel[col:col+BM, 2] = 255
    colorwheel[col:col+BM, 0] = np.transpose(np.floor(255*np.arange(0, BM) / BM))
    col += + BM

    # MR
    colorwheel[col:col+MR, 2] = 255 - np.transpose(np.floor(255 * np.arange(0, MR) / MR))
    colorwheel[col:col+MR, 0] = 255

    return colorwheel

Save Flo files as images

In [ ]:
if visualize == True:

  import os
  import PIL.Image
  def mkdir_ifnotexists(dir):
      if os.path.exists(dir):
          return
      os.mkdir(dir)

  if no_average_frames != None:
    directory = "./Average_Frames"

  elif running_average == True:
    directory = "./Running_Avg_Frames"

  else:
    directory = '/content/flownet2pytorch/output/inference/run.epoch-0-flow-field/'

  # flo_pth='/content/flownet2pytorch/output/inference/run.epoch-0-flow-field/'
  flo_pth = directory
  flos=[flo_pth + f for f in os.listdir(flo_pth)]
  mkdir_ifnotexists('./FlowFrames')
  length = len(flos)
  for i in range(length):
    if flos[i][-3:] == "flo":
      print(i+1, "/", length)
      PIL.Image.fromarray(flow_to_image(read_flow(flos[i]))).save('./FlowFrames/'+os.path.basename(flos[i])+'.png')
      os.remove(flos[i])

Generate video from Flo

In [ ]:
if visualize == True:
  os.system('ffmpeg -r 25 -i FlowFrames/%6d.flo.png -vcodec libx264 -b 10M -y FlowVideo.mp4')

  print("My program took", time.time() - start_time, "to run")

In [ ]:
# from google.colab import files
# files.download('FlowVideo.mp4')

In [ ]:
if visualize == True:

  from IPython.display import HTML
  from base64 import b64encode
  mp4 = open('FlowVideo.mp4','rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  HTML("""
  <video width=400 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

# Head Coordinates

In [ ]:
%cd /content/

frames = []
dir = 'NWPU_Results/'

for file in sorted(os.listdir('./NWPU_Results')):
  if file[-3:] == 'csv':
    frames.append(dir + file)

/content


In [ ]:
# Import
import numpy as np
from sklearn.cluster import KMeans

f = open("temp.txt", 'w')

# Set Values
threshold = 0.5 * (10**(-1))
weight = 2

for i, frame in enumerate(frames[1:]):
    print(i, " / ", len(frames)-1)

    # Get Excel Sheet
    X = np.genfromtxt(frame, delimiter=',')

    os.remove(frame)

    points_val = X

    print(X.shape)

    # First value is always nan for some reason
    X[0][0] = 0

    # Total Value
    total_val = np.sum(X)
    number_people = int(round(np.sum(X)/100))
    print("Number of people:", number_people)

    vals = X[X[:,:] > threshold] # Contains the values. 1D array

    index = np.argwhere(points_val > threshold) # Contains the index. 2D array.

    # Making a 2D Array with point, val -> (x, y, val)
    list_for_cluster = [] # Contains a 2D array with point, val

    for i, point in enumerate(index):
        val = vals[i]
        list_for_cluster.append(np.append(point, val))

    final_list = np.array(list_for_cluster) # Contains a 2D array with point, val -> (x, y, val)
    final_list.shape

    x_index = index[:,0]
    y_index = index[:,1]

    cluster_vals = index

    weights = vals ** weight

    # Cluster based on K Means Clustering
    # Forcefully put how many ever people we found, make that many clusters

    kmeans = KMeans(
        init="random",
        n_clusters=number_people,
        n_init=10,
        max_iter=500,
        random_state=42
    )

    kmeans.fit(cluster_vals, sample_weight = weights)

    kmeans.inertia_
    kmeans.cluster_centers_
    kmeans.n_iter_
    y_kmeans = kmeans.predict(cluster_vals)

    centers = kmeans.cluster_centers_

    # Write to cords.txt
    line = ""
    for center in centers:
        line += str(int(center[1])) + " " + str(int(center[0])) + " "

    line = line.strip()
    line += "\n "

    f.write(line)

f.close()


0  /  90
(1024, 1920)
Number of people: 34
1  /  90
(1024, 1920)
Number of people: 32
2  /  90
(1024, 1920)
Number of people: 21
3  /  90
(1024, 1920)
Number of people: 12
4  /  90
(1024, 1920)
Number of people: 11
5  /  90
(1024, 1920)
Number of people: 11
6  /  90
(1024, 1920)
Number of people: 15
7  /  90
(1024, 1920)
Number of people: 14
8  /  90
(1024, 1920)
Number of people: 13
9  /  90
(1024, 1920)
Number of people: 13
10  /  90
(1024, 1920)
Number of people: 14
11  /  90
(1024, 1920)
Number of people: 13
12  /  90
(1024, 1920)
Number of people: 13
13  /  90
(1024, 1920)
Number of people: 14
14  /  90
(1024, 1920)
Number of people: 22
15  /  90
(1024, 1920)
Number of people: 16
16  /  90
(1024, 1920)
Number of people: 15
17  /  90
(1024, 1920)
Number of people: 18
18  /  90
(1024, 1920)
Number of people: 17
19  /  90
(1024, 1920)
Number of people: 20
20  /  90
(1024, 1920)
Number of people: 14
21  /  90
(1024, 1920)
Number of people: 19
22  /  90
(1024, 1920)
Number of people: 2

In [ ]:
cords = open("cords.txt", 'w')
temp = open("temp.txt", 'r')
temp_lines = temp.read().split("\n")

for i, line in enumerate(temp_lines):
  fline = line.strip()
  if i != len(temp_lines) - 1:
    fline += "\n"
  cords.write(fline)

cords.close()
temp.close()

# Clustering

## Manual Annotation



1.   Take any frame from the folder frames. RENAME it to 'image.png'
2.   Run the file "annotate.py" from the github folder
3.   Paste the output text file "text_speed.txt" in the same folder as the "cords.txt". This is the /content/ directory




HOW TO RUN ANNOTATE.PY

0.   Keep the annotate.py and the first frame in the same directory and run 'python annotate.py'
1.   Annotate the first and second x meters
2.   Annotate the road in a clockwise fashion starting from where you marked the first x meters
3.   Fil these variables:

  *   temp_approx_meters_p1 = 0.5 # How many meters the first x meters represents
  *   temp_approx_meters_p2 = 0.5 # How many meters the second y meters represents
  *   normalize_meters = 1 # If you want the output to be represented in some other amount of meters, do that here. For eg. if you have annotated first point as 0.5m and second as 1m and want the output to be catered to by 1m, then make this 1.


In [ ]:
# Setup
!mkdir Clustering

!mv cords.txt text_speed.txt Clustering

%cd Clustering

!mkdir text_files
!mkdir output_frames

!mv cords.txt text_speed.txt text_files

mv: cannot stat 'text_speed.txt': No such file or directory
/content/Clustering
mv: cannot stat 'text_speed.txt': No such file or directory


## Cluster

In [ ]:
!git clone https://github.com/JibKh/NVIDIA-FlowNet2-Google-Colab.git

Cloning into 'NVIDIA-FlowNet2-Google-Colab'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 107 (delta 49), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (107/107), 48.13 MiB | 12.88 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [ ]:
%cd NVIDIA-FlowNet2-Google-Colab/
!mv cluster.py /content/Clustering
%cd ..
!rm -r NVIDIA-FlowNet2-Google-Colab/

/content/Clustering/NVIDIA-FlowNet2-Google-Colab
/content/Clustering


In [ ]:
# Have to remove first frame and rename
!rm /content/frames/000000.png


file_dir = "/content/frames/"
i = 0
for count, filename in enumerate(sorted(os.listdir(file_dir))):
  # print(filename)
  if filename[-3:] == "png":
    src = file_dir + filename
    dst = file_dir + str(i).zfill(6) + '.png'
    os.rename(src, dst)
    i += 1


In [ ]:
!python cluster.py

Traceback (most recent call last):
  File "cluster.py", line 496, in <module>
    main()
  File "cluster.py", line 486, in main
    points_labels, velocity_labels, speed_labels, average_speed, total_average_speed = cluster_2()
  File "cluster.py", line 262, in cluster_2
    total_total += total_average_speed
NameError: name 'total_total' is not defined
